## Inspired and referred from Andrej Karpathy's article of RNNs

In [4]:
import numpy as np

# data I/O
data = open('input.txt', 'r').read() # should be simple plain text file
chars = list(set(data))
data_size, vocab_size = len(data), len(chars)
print ('data has %d characters, %d unique.' % (data_size, vocab_size))
char_to_ix = { ch:i for i,ch in enumerate(chars) }
ix_to_char = { i:ch for i,ch in enumerate(chars) }

data has 25 characters, 14 unique.


In [7]:
hidden_size = 100
seq_length = 24
learning_rate = 1e-1

Wxh = np.random.randn(hidden_size, vocab_size) * 0.01
Whh = np.random.randn(hidden_size, hidden_size) * 0.01
Why= np.random.randn(vocab_size, hidden_size) * 0.01
bh = np.zeros((hidden_size, 1))
by = np.zeros((vocab_size, 1))


In [8]:
# Defining the loss function


def loss_fn(inputs, targets, h_ini):
    """Compute loss and gradients for a sequence.

    inputs, targets are both lists of integers.
    h_ini is the initial hidden state (column vector).
    Returns the loss, gradients for parameters, and the last hidden state.
    """
    xs, hs, ys, ps = {}, {}, {}, {}
    hs[-1] = np.copy(h_ini)
    loss = 0.0

    # Ensure inputs and targets have matching lengths
    assert len(inputs) == len(targets), f"inputs and targets length mismatch: {len(inputs)} vs {len(targets)}"
    T = len(targets)

    # Forward pass
    for t in range(T):
        # One-hot encode input
        xs[t] = np.zeros((vocab_size, 1))
        xs[t][inputs[t]] = 1

        # Hidden state
        hs[t] = np.tanh(np.dot(Wxh, xs[t]) + np.dot(Whh, hs[t - 1]) + bh)

        # Unnormalized log probabilities for next chars (logits)
        ys[t] = np.dot(Why, hs[t]) + by

        # Softmax probabilities for next chars
        ps[t] = np.exp(ys[t]) / np.sum(np.exp(ys[t]))

        # Cross-entropy loss for this step
        loss += -np.log(ps[t][targets[t], 0])

    # Backward pass: compute gradients going backwards
    dWxh, dWhh, dWhy = np.zeros_like(Wxh), np.zeros_like(Whh), np.zeros_like(Why)
    dbh, dby = np.zeros_like(bh), np.zeros_like(by)
    dhnext = np.zeros_like(hs[0])

    for t in reversed(range(T)):
        dy = np.copy(ps[t])
        dy[targets[t]] -= 1  # gradient of loss w.r.t. logits
        dWhy += np.dot(dy, hs[t].T)
        dby += dy

        # Backprop into hidden state
        dh = np.dot(Why.T, dy) + dhnext
        dhraw = (1 - hs[t] * hs[t]) * dh  # backprop through tanh
        dbh += dhraw
        dWxh += np.dot(dhraw, xs[t].T)
        dWhh += np.dot(dhraw, hs[t - 1].T)
        dhnext = np.dot(Whh.T, dhraw)

    # Clip gradients to mitigate exploding gradients
    for dparam in [dWxh, dWhh, dWhy, dbh, dby]:
        np.clip(dparam, -5, 5, out=dparam)

    return loss, dWxh, dWhh, dWhy, dbh, dby, hs[len(inputs) - 1]



def sample(h, seed_ix, n):
    """Sample a sequence of integers from the model.

    h is memory state, seed_ix is seed letter for first time step.
    """
    x = np.zeros((vocab_size, 1))
    x[seed_ix] = 1
    ixes = []

    for t in range(n):
        h = np.tanh(np.dot(Wxh, x) + np.dot(Whh, h) + bh)
        y = np.dot(Why, h) + by
        p = np.exp(y) / np.sum(np.exp(y))
        ix = np.random.choice(range(vocab_size), p=p.ravel())
        x = np.zeros((vocab_size, 1))
        x[ix] = 1
        ixes.append(ix)

    return ixes


# Training loop
n, p = 0, 0
mWxh, mWhh, mWhy = np.zeros_like(Wxh), np.zeros_like(Whh), np.zeros_like(Why)
mbh, mby = np.zeros_like(bh), np.zeros_like(by)  # memory variables for Adagrad
smooth_loss = -np.log(1.0 / vocab_size) * seq_length  # loss at iteration 0

while True:
    # Prepare inputs (sweeping left to right in steps of seq_length)
    if p + seq_length + 1 >= len(data) or n == 0:
        hprev = np.zeros((hidden_size, 1))  # reset RNN memory
        p = 0  # go from start of data

    inputs = [char_to_ix[ch] for ch in data[p : p + seq_length]]
    targets = [char_to_ix[ch] for ch in data[p + 1 : p + seq_length + 1]]

    # Sample from the model now and then
    if n % 100 == 0:
        sample_ix = sample(hprev, inputs[0], 200)
        txt = "".join(ix_to_char[ix] for ix in sample_ix)
        print("----\n%s\n----" % txt)

    # Forward seq_length characters through the net and fetch gradient
    loss, dWxh, dWhh, dWhy, dbh, dby, hprev = loss_fn(inputs, targets, hprev)
    smooth_loss = smooth_loss * 0.999 + loss * 0.001
    if n % 100 == 0:
        print("iter %d, loss: %f" % (n, smooth_loss))

    # Perform parameter update with Adagrad
    for param, dparam, mem in zip(
        [Wxh, Whh, Why, bh, by],
        [dWxh, dWhh, dWhy, dbh, dby],
        [mWxh, mWhh, mWhy, mbh, mby],
    ):
        mem += dparam * dparam
        param += -learning_rate * dparam / np.sqrt(mem + 1e-8)  # Adagrad update

    p += seq_length  # move data pointer
    n += 1  # iteration counter

----
ehe m ,,amh htsto maeyHiHheeHoymHiyity e nss soe,Hnty   tH,eyHaymts,ltnlmaHeo yoansHmnlyHshhhleyeim,eho,ooaimam,Hos ,oioaalnaHHmylonHo athyh m tmalitehyonalmyainoi ste aaoaotyiyhmayih tm,lHHs,ntmh,m y
----
iter 0, loss: 63.337361
----
ello, my nshmy name is Het nshetis HetanshetansHmtanshmy sshetanshetanshetanshey nshmianshen nshetanshetansmetanshetanametaname is Hetansheten hetaname is He is Httanametanshetanshmtanshetanshetanshet
----
iter 100, loss: 59.311916
----
ello, my name is Hetanshetaname is Hetansmetanshelia HetanshetanshetanshetansHetanshey name is Hetanshey name is Hetanshetanshetanshetanshey name isshetanshetanshetanshetanshetanshetanshet nshe, name 
----
iter 200, loss: 53.800611
----
ello, my naHe is Hetanshetanshetanshetanshmtanshetanshetanshetanshetanshetanshetanshetanshetanshetanshehanshetanshetanshetis Hetanshetanahe is Hetanshetanshetanshetanshetanshetanshetanshetanshetanshet
----
iter 300, loss: 48.717585
----
ello, my name is Hetanshetanshetanshetanshetanshe

KeyboardInterrupt: 